In [1]:
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import urllib.request
import requests
import pandas as pd
import re
import csv

def crawl_page(link=''):
    # req = urllib.request.Request(
    #         link, 
    #         data=None, 
    #         headers={
    #             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    #             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393'
    #         }
    # )
    # try:
    #     page= urllib.request.urlopen(req)
    #     return_code= page.getcode()
    #     soup_page= BeautifulSoup(page,'html.parser')
    #     return {'page':soup_page, 'status': return_code}
    # except HTTPError as err:
    #     if err.code == 404:
    #         return_code=404
    #         return {'page':'no page', 'status': return_code}
    #     else:
    #         raise
    try:
        response = requests.get(
            url="http://api.scraperapi.com", #scrapeapi
            params={
                'api_key': 'b4b2907633a4c7758d3e81f40c85a442',
                'url': link,  
            },
            
        )
        return_code= 200
        soup_page= BeautifulSoup(response.content,'html.parser')
        return {'page':soup_page, 'status': return_code}
    except requests.exceptions.HTTPError as err:
        if err.response.status_code == 404:
            return_code=404
            return {'page':'no page', 'status': return_code}
    

def write_csv(line=[], file=''):
    with open(file, 'a', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=' ',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(line)

def remove_row_csv():
    with open('temp_27.csv') as inp:
        for link in csv.reader(inp):
            count=0
            temp=link
            with open('used_estate.txt') as out:
                for estate in csv.reader(out):
                    if re.match("(.*)/"+estate[0],temp[1]) is not None:
                        count+=1
            if count==0:
                new_row= [link[0]+','+link[1]]
                write_csv(new_row,"temp_28.csv")

def create_urls(file=''):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        for row in csv_reader:
            new_row= [str(line_count)+','+row[0]]
            write_csv(new_row,'property_url_v4.csv')
            line_count+=1

    

In [10]:
class DataCollect():
    def __init__(self):
        self.__url_to_open='https://www.property24.com'
        self.__soup=crawl_page(self.__url_to_open)['page']
        self.__area_url=[]
        self.__list_url=[]
        self.__urls=pd.read_csv("temp_59_v4.csv",usecols= ['id','url']) 
    
    def collect_all_first_page_url(self):
        other_links= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all('a',class_='p24_bold')
        for link in other_links:
            self.__area_url.append(self.__url_to_open+link.get('href'))
        first_element_tag= self.__soup.find('div',class_='p24_popular').find(class_='col-8').find_all(class_='row')[1]
        first_link=first_element_tag.find('a').get('href')
        temp_soup= crawl_page(self.__url_to_open+first_link)['page']
        parent_link= temp_soup.find(id='breadCrumbContainer').find_all('a')[1]
        self.__area_url.append(self.__url_to_open+parent_link.get('href'))
    
    def collect_all_url(self):
        self.collect_all_first_page_url()
        for link in self.__area_url:
            soup= crawl_page(link)['page']
            link_house_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("houses-for-sale")).get('href')
            link_apart_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("apartments-for-sale")).get('href')
            link_townhouse_tag= soup.find('ul',class_='p24_relatedSales').find(href=re.compile("townhouses-for-sale")).get('href')
            self.__list_url.append(link_house_tag)
            self.__list_url.append(link_apart_tag)
            self.__list_url.append(link_townhouse_tag)

    def get_property_data(self, link=''):
        data={}

        estate_id= link.split('/')[-1].split('?')[0]

        prop_soup_element= crawl_page(self.__url_to_open+link)

        
        if prop_soup_element['status'] !=200:
            print('heres')
            return False
        
        prop_soup= prop_soup_element['page']

        if prop_soup is None:
            print('here')
            return False

        #check if page not found
        check_page_not_found_msg= prop_soup.select_one('.p24_not_found')
        if check_page_not_found_msg is not None:
            print("====> Page not found !!")
            return False
        
        #check page expired
        check_page_expired = prop_soup.select_one('.p24_results.p24_expired')
        if check_page_expired is not None:
            print("====> Page expired !!")
            return False
        

        name= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('h1').get_text()
        data['name']=name

        price= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find('div',class_="p24_price").get_text()
        data['price']=price

        province= link.split('/')[4]
        data['province']=province

        city= link.split('/')[3]
        data['city']=city


        is_adress= len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) >= 3
        if is_adress:
            if len(prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")) == 3:
                index=-1
            else:
                index=3
            address= prop_soup.find('div',class_='p24_listingFeaturesWrapper').find_all('div',class_="p24_mBM")[index].get_text()
            data['address']=address
        else:
            address=  prop_soup.select_one('.p24_addressPropOverview').get_text()
            data['address']=address

        #property overview
        if type(prop_soup.select_one('.p24_propertyOverview')) is not None:
            prop_overview_panel= prop_soup.find('div',class_='p24_propertyOverview').find('div',class_='panel-body')
            prop_overview_row= prop_overview_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #rooms
        rooms_panel=prop_soup.select_one('#js_accordion_rooms')
        if rooms_panel is not None:
            prop_overview_row= rooms_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #external_features
        external_features_panel=prop_soup.select_one('#js_accordion_externalfeatures')
        if external_features_panel is not None:
            prop_overview_row= external_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value
        
        #other_features
        other_features_panel=prop_soup.select_one('#js_accordion_otherfeatures')
        if other_features_panel is not None:
            prop_overview_row= other_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #building_features
        building_features_panel=prop_soup.select_one('#js_accordion_building')
        if building_features_panel is not None:
            prop_overview_row= building_features_panel.find_all('div',class_='p24_propertyOverviewRow')
            for prop in prop_overview_row:
                key= prop.find('div',class_='p24_propertyOverviewKey').get_text()
                # value= prop.select_one('.noPadding > .p24_info').get_text()
                value= prop.find('div',class_='noPadding').contents[1].get_text()
                data[key]=value

        #points_interest
        interest_soup= crawl_page(self.__url_to_open+'/ListingReadOnly/PointsOfInterestForListing?ListingNumber=P24-'+estate_id)
        # points_interest_panel=prop_soup.select_one('#accordian-points-of-interest')
        # print(points_interest_panel)
        if interest_soup['status'] == 200:
            categories= interest_soup['page'].select('.js_P24_POICategory')
            for cat in categories:
                interest= cat.select_one('.row > .col-6 > .p24_semibold')
                distances= cat.select('.row > .col-6.noPadding.p24_semibold')
                dist_tab=[]
                for dist in distances:
                    dist_tab.append(dist.get_text())
                data[interest.get_text()]=dist_tab

        print(str(data))
        write_csv([str(data)],'data_property_v4.csv')

        
        return True

    def crawl_property_per_page(self,city,page,number_properties):
        link=self.__url_to_open+city+'/p'+str(page)
        print('==>Page link: ',link)
        cur_soup= crawl_page(link)['page']
        properties= cur_soup.find_all('div',class_='js_resultTile')
        for prop_index in range(len(properties)-1):
            prop_link= properties[prop_index].find('a').get('href')
            write_csv([prop_link],'data_property_url.csv')

        number_properties+= len(properties)
        return number_properties

    def crawl_property_per_city(self,city,cur_page):
        soup= crawl_page(self.__url_to_open+city)['page']
        pagination= soup.find('div',class_='p24_pager').find_all('li')[-1].find('a').get('data-pagenumber')
        page_number= int(pagination)
        number_properties=0
        print("City url: ",city)
        for page in range(cur_page, page_number+1):
            print("=>Page number: ",page)
            number_properties = self.crawl_property_per_page(city,page,number_properties)
        print(number_properties)
    
    def crawl_properties(self):
        #shuffle
        self.__urls = self.__urls.sample(frac = 1).reset_index(drop=True)
        for i in range(self.__urls.size):
            prop_link= self.__urls.iloc[i]['url']
            
            print('==>Property link: ', prop_link)
            check_property_crawl = self.get_property_data(prop_link)
            if check_property_crawl:
                self.__urls= self.__urls.drop(self.__urls.index[i])
                self.__urls.to_csv('temp_60_v4.csv',index=False)

    def crawl_property_url(self):
        self.__list_url=['/townhouses-for-sale/gauteng/1']
        for link in self.__list_url:
            self.crawl_property_per_city(link,1)

In [11]:
dc= DataCollect()
try:
    dc.crawl_properties()
except:
    dc.crawl_properties()

==>Property link:  /for-sale/groenkloof/pretoria/gauteng/3955/110075428
{'name': '3 Bedroom House for Sale in Groenkloof', 'price': 'R 3 500 000', 'province': 'gauteng', 'city': 'pretoria', 'address': 'Groenkloof, Pretoria', 'Listing Number': '110075428', 'Type of Property': 'House', 'Lifestyle': 'Gated Community', 'Listing Date': '09 June 2021', 'Erf Size': '1 000 m²', 'Pets Allowed': 'Yes', 'Bedrooms': '3', 'Bathrooms': '2.5', 'Kitchen': 'Laundry, Stove, Dishwasher Connection', 'Reception Rooms': '1', 'Dining Room': '1', 'Rooms': 'Patio, Blinds', 'Garden': 'Yes', 'Security': 'Electric Garage, Electric Gate, 24 Hour Response, Burglar Bars, Electric fencing, Boomed Area', 'Special Feature': 'Built in Braai', 'Window': 'Wood', 'Floor': 'Carpets, Tiled Floors, Laminated Floors', 'Transport and Public Services': ['0.63km', '0.81km', '1.13km', '1.15km', '1.33km'], 'Food and Entertainment': ['0.74km', '0.80km', '0.94km', '0.96km', '1.07km', '1.07km', '1.08km', '1.08km', '1.08km', '1.09km', 

{'name': '4 Bedroom House for Sale in New Redruth', 'price': 'R 3 200 000', 'province': 'gauteng', 'city': 'alberton', 'address': '\n4 Bedroom House for Sale in New Redruth\n', 'Listing Number': '109711509', 'Type of Property': 'House', 'Street Address': ' St Michael Street, New Redruth', 'Listing Date': '14 March 2021', 'Erf Size': '2 000 m²', 'Rates and Taxes': 'R 3 200', 'Pets Allowed': 'Yes', 'Bedrooms': '4', 'Bathrooms': '3', 'Garage': '2', 'Parking': '2', 'Garden': 'Yes', 'Pool': 'Yes', 'Internet Access': 'Fibre', 'Education': ['0.25km', '0.49km', '1.21km', '1.27km', '1.51km', '1.71km', '1.72km', '1.78km', '1.94km', '1.98km', '2.36km', '3.12km', '3.33km', '3.39km', '3.51km', '3.98km', '4.46km', '4.47km', '4.52km', '4.93km', '5.02km'], 'Shopping': ['0.38km'], 'Transport and Public Services': ['2.48km', '3.61km', '3.62km', '4.16km', '4.36km', '4.50km', '4.82km'], 'Sports and Leisure': ['4.27km']}
==>Property link:  /for-sale/rhodesfield/kempton-park/gauteng/1355/109996381
====> Pag

{'name': '5 Bedroom House for Sale in Midstream Ridge Estate', 'price': 'R 7 475 000', 'province': 'gauteng', 'city': 'centurion', 'address': '\n5 Bedroom House for Sale in Midstream Ridge Estate\n', 'Listing Number': '109832353', 'Type of Property': 'House', 'Listing Date': '08 April 2021', 'Erf Size': '1 000 m²', 'Floor Size': '500 m²', 'Levies': 'R 1 970', 'Rates and Taxes': 'R 3 500', 'Bedrooms': '5', 'Bathrooms': '4.5', 'Garage': '3', 'Education': ['0.82km', '2.87km'], 'Transport and Public Services': ['0.92km', '0.98km', '2.62km', '4.21km'], 'Health': ['0.98km', '3.80km'], 'Food and Entertainment': ['1.00km', '1.00km', '1.01km', '3.68km', '3.70km', '3.74km', '3.75km', '3.78km', '3.80km', '3.80km', '3.81km', '3.81km', '3.83km', '4.19km', '4.20km', '4.22km', '4.23km', '4.24km', '4.25km'], 'Sports and Leisure': ['1.31km'], 'Shopping': ['3.79km', '4.21km']}
==>Property link:  /for-sale/reyno-ridge/witbank/mpumalanga/2533/109779850
{'name': '2 Bedroom Apartment / Flat for Sale in Reyn

{'name': '1 Bedroom Apartment / Flat for Sale in Sunnyside', 'price': 'R 275 000', 'province': 'gauteng', 'city': 'pretoria', 'address': 'Sunnyside, Pretoria', 'Listing Number': '109373991', 'Type of Property': 'Apartment / Flat', 'Listing Date': '20 November 2020', 'Erf Size': '1 075 m²', 'Floor Size': '41 m²', 'Price per m²': 'R 6 707', 'Levies': 'R 964', 'Rates and Taxes': 'R 10', 'Bedroom': 'Bedroom', 'Bathroom': 'Bathroom', 'Kitchen': 'Kitchen', 'Reception Rooms': '1', 'Lounge': 'Lounge', 'Entrance Hall': 'Entrance Hall', 'Garden': 'Garden', 'Shopping': ['0.04km', '0.13km', '0.30km', '0.41km', '0.73km'], 'Health': ['0.11km', '0.53km', '0.57km', '0.70km'], 'Transport and Public Services': ['0.12km', '0.17km', '0.19km', '0.24km', '0.48km', '0.51km', '0.74km', '0.74km'], 'Food and Entertainment': ['0.16km', '0.17km', '0.35km', '0.40km', '0.45km', '0.48km', '0.55km', '0.70km', '0.72km'], 'Sports and Leisure': ['0.33km', '0.77km'], 'Education': ['0.44km', '0.53km']}
==>Property link:  

{'name': '2 Bedroom House for Sale in Bela Bela', 'price': 'R 525 000', 'province': 'limpopo', 'city': 'bela-bela', 'address': 'Bela Bela', 'Listing Number': '109874520', 'Type of Property': 'House', 'Listing Date': '17 April 2021', 'Levies': 'R 738', 'Pets Allowed': 'Yes', 'Bedrooms': '2', 'Bathrooms': '1.5', 'Kitchens': '1', 'Lounges': '1', 'Parking': '1', 'Garden': 'Yes', 'Education': ['0.12km', '0.53km', '0.79km', '0.98km', '1.57km', '1.99km', '2.03km', '2.08km', '2.65km', '2.80km', '2.91km', '3.26km', '3.34km', '4.82km', '7.02km', '7.42km', '7.99km'], 'Transport and Public Services': ['1.03km'], 'Sports and Leisure': ['2.70km']}
==>Property link:  /for-sale/willowmore/willowmore/eastern-cape/7587/109782509
{'name': 'Apartment / Flat for Sale in Willowmore', 'price': 'R 875 000', 'province': 'eastern-cape', 'city': 'willowmore', 'address': 'Willowmore', 'Listing Number': '109782509', 'Type of Property': 'Apartment / Flat', 'Listing Date': '23 March 2021', 'Erf Size': '467 m²', 'Flo

{'name': '3 Bedroom House for Sale in Rivonia', 'price': 'R 4 300 000', 'province': 'gauteng', 'city': 'sandton', 'address': '\n3 Bedroom House for Sale in Rivonia\n', 'Listing Number': '109792919', 'Type of Property': 'House', 'Street Address': '12 8th Avenue, Rivonia', 'Description': 'Single Storey', 'Listing Date': '25 March 2021', 'Levies': 'R 2 000', 'Rates and Taxes': 'R 2 562', 'Pets Allowed': 'Yes', 'Bedrooms': '3', 'Bathrooms': '2', 'Kitchen': '1', 'Office': '1', 'Domestic Rooms': '1', 'Reception Rooms': '1', 'Lounge': '2', 'Dining Room': '1', 'Family/TV Room': '1', 'Entrance Hall': '1', 'Rooms': 'Patio', 'Parking': '9', 'Garden': '1', 'Pool': '1', 'Outbuilding': '1', 'Security': 'Security Gate, Intercom, Closed Circuit TV, 24 Hour Access, Guard House, Electric fencing', 'Special Feature': 'Borehole, Balcony', 'Flatlet': 'Yes', 'Backup Water': 'Yes', 'Roof': 'Thatch', 'Shopping': ['0.52km', '0.52km', '0.59km', '0.70km'], 'Transport and Public Services': ['0.57km', '0.61km', '0

{'name': '3 Bedroom Apartment / Flat for Sale in Waterfall', 'price': 'R 1 900 000', 'province': 'gauteng', 'city': 'midrand', 'address': 'Waterfall, Midrand', 'Listing Number': '109962289', 'Type of Property': 'Apartment / Flat', 'Listing Date': '11 May 2021', 'Floor Size': '114 m²', 'Price per m²': 'R 16 667', 'Levies': 'R 1 368', 'No Transfer Duty': 'Yes', 'Rates and Taxes': 'R 700', 'Pets Allowed': 'No', 'Bedrooms': '3', 'Bathrooms': '2', 'Kitchens': '1', 'Lounges': '1', 'Dining Rooms': '1', 'Parking': '2', 'Pool': 'Yes', 'Health': ['2.22km', '3.59km'], 'Transport and Public Services': ['2.59km', '2.83km', '2.84km', '3.73km'], 'Food and Entertainment': ['2.84km', '2.86km', '2.86km', '2.91km', '3.58km', '3.58km', '3.58km', '3.59km', '3.59km', '3.70km'], 'Education': ['2.87km', '2.96km', '3.05km', '3.09km', '3.36km', '3.59km', '3.72km', '3.73km'], 'Shopping': ['3.58km', '3.58km', '3.58km', '3.66km', '3.68km'], 'Sports and Leisure': ['3.62km']}
==>Property link:  /for-sale/meyerville/

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))